In [ ]:
!pip install clarifaipyspark
!pip install protobuf==4.24.2

# Testing ClarifaiPySpark SDK

### Setting Env variables

In [ ]:
import os
from clarifaipyspark.client import ClarifaiPySpark

os.environ['CLARIFAI_PAT'] = ''

2023-10-23 09:04:36 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=489096;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=194774;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Creating ClarifaiPyspark object & creating/fetching image dataset from app

In [ ]:
cspark_obj = ClarifaiPySpark(user_id='mansi_k', app_id='databricks_tester_img')

dataset_obj = cspark_obj.dataset(dataset_id='dataset1')

2023-10-23 09:04:40 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=498902;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=634484;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Fetching current inputs from image dataset

In [ ]:
my_inputs = list(dataset_obj.list_inputs())
my_inputs

2023-10-23 09:07:10 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=653055;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=627783;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

[id: "dataset1-train-3647386"
 data {
   image {
     url: "https://data.clarifai.com/orig/users/mansi_k/apps/databricks_tester_img/inputs/image/1249d3d8c8157d1b2dcdb2561175374a"
     hosted {
       prefix: "https://data.clarifai.com"
       suffix: "users/mansi_k/apps/databricks_tester_img/inputs/image/1249d3d8c8157d1b2dcdb2561175374a"
       sizes: "orig"
       sizes: "tiny"
       sizes: "small"
       sizes: "large"
       crossorigin: "use-credentials"
     }
     image_info {
       width: 512
       height: 288
       format: "JPEG"
       color_mode: "YUV"
     }
   }
   concepts {
     id: "id-hamburger"
     name: "hamburger"
     value: 1
     app_id: "databricks_tester_img"
   }
 }
 created_at {
   seconds: 1698052012
   nanos: 477555000
 }
 modified_at {
   seconds: 1698052013
   nanos: 977120000
 }
 status {
   code: INPUT_DOWNLOAD_SUCCESS
   description: "Download complete"
 },
 id: "dataset1-train-3520891"
 data {
   image {
     url: "https://data.clarifai.com/orig/u

### Upload images from volume folder to Clarifai dataset

In [ ]:
dataset_obj.upload_dataset_from_folder(folder_path='/Volumes/mansi_test/default/cat', input_type='image', labels=True)

2023-10-23 08:27:37 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=55296;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=786623;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading inputs: 100%|██████████| 1/1 [00:05<00:00,  5.36s/it]


### Export image annotations to spark dataframe

In [ ]:
annot_df = dataset_obj.export_annotations_to_dataframe()
annot_df

2023-10-23 08:30:22 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=777205;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=86519;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

DataFrame[annotation: map<string,array<map<string,string>>>, created_at: string, id: string, input_id: string, modified_at: string, user_id: string]

### Write annotations dataframe to volume as delta table

In [ ]:
annot_df.write.format("delta").mode("overwrite").save("/Volumes/mansi_test/default/test_vol1/imgAnnsDeltaTable1") 


2023-10-23 08:30:51 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=660796;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=177253;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Load annotations delta table from volume

In [ ]:
df_delta = spark.read.format("delta").load("/Volumes/mansi_test/default/test_vol1/imgAnnsDeltaTable1")
df_delta.show()

2023-10-23 08:31:11 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=857207;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=734426;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

+--------------------+-------------------+--------------------+--------+-------------------+-------+
|          annotation|         created_at|                  id|input_id|        modified_at|user_id|
+--------------------+-------------------+--------------------+--------+-------------------+-------+
|{concepts -> [{na...|10/23/% 08:27:   %5|da7eb3483a654a42a...|      c5|10/23/% 08:27:   %5|mansi_k|
|{concepts -> [{na...|10/23/% 08:27:   %5|f65d75b3446b40ffa...|     c11|10/23/% 08:27:   %5|mansi_k|
|{concepts -> [{na...|10/23/% 08:27:   %5|8fd99dc05eef46849...|      c1|10/23/% 08:27:   %5|mansi_k|
|{concepts -> [{na...|10/23/% 08:27:   %5|9426763104ea42fbb...|      c7|10/23/% 08:27:   %5|mansi_k|
|{concepts -> [{na...|10/23/% 08:27:   %5|d93bb8387f6743b8a...|      c8|10/23/% 08:27:   %5|mansi_k|
|{concepts -> [{na...|10/23/% 08:27:   %5|aed44152eaad4fbcb...|      c6|10/23/% 08:27:   %5|mansi_k|
|{concepts -> [{na...|10/23/% 08:27:   %5|b3847d13486f4f118...|      c2|10/23/% 08:27:   %5

### Loop through delta table rows & columns

In [ ]:
for row in df_delta.collect()[:4]:
    print(row['id'])
    print(row['input_id'])
    print(row['user_id'])
    print(row['created_at'])
    print(row['input_id'])
    print(row['annotation'])
    print("########################\n")

2023-10-23 08:31:26 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=698390;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=992550;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

da7eb3483a654a42ace1ebac76a69fdc
c5
mansi_k
10/23/% 08:27:   %5
c5
{'concepts': [{'name': 'cat', 'id': 'id-cat', 'value': '1.0', 'appId': 'databricks_tester_img'}]}
########################

f65d75b3446b40ffa012735a31dcbb62
c11
mansi_k
10/23/% 08:27:   %5
c11
{'concepts': [{'name': 'cat', 'id': 'id-cat', 'value': '1.0', 'appId': 'databricks_tester_img'}]}
########################

8fd99dc05eef468492623153531700c9
c1
mansi_k
10/23/% 08:27:   %5
c1
{'concepts': [{'name': 'cat', 'id': 'id-cat', 'value': '1.0', 'appId': 'databricks_tester_img'}]}
########################

9426763104ea42fbb44ff73677fd3806
c7
mansi_k
10/23/% 08:27:   %5
c7
{'concepts': [{'name': 'cat', 'id': 'id-cat', 'value': '1.0', 'appId': 'databricks_tester_img'}]}
########################



### Testing export_images_to_volume()

In [ ]:
dataset_obj.export_images_to_volume(path="/Volumes/mansi_test/default/test_vol1", input_response=my_inputs[:5])

2023-10-23 08:31:44 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=244234;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=125648;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Load image csv file & upload text data into app using dataframe

In [ ]:
spark = SparkSession.builder.appName("Clarifai-spark").getOrCreate()
df = spark.read.option("header",True).csv("/Volumes/mansi_test/default/test_vol1/img_data2.csv")
df.show()

2023-10-23 09:02:37 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=783896;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=96789;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

+-------+--------------------+--------+
|inputid|               input|concepts|
+-------+--------------------+--------+
|  img11|https://img.freep...|     dog|
|  img21|https://images.un...|    duck|
|  img31|https://mymodernm...|    duck|
+-------+--------------------+--------+



In [ ]:
dataset_obj.upload_dataset_from_dataframe(dataframe=df, input_type='image', df_type='url', labels=True)

2023-10-23 09:02:44 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=315145;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=593548;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading inputs:   0%|          | 0/1 [00:00<?, ?it/s]

2023-10-23 09:02:45 WARNING  WARNING:clarifai.client.input:code: INPUT_DUPLICATE                       ]8;id=742428;file:///local_disk0/.ephemeral_nfs/envs/pythonEnv-07884fc6-4bdc-4cb9-bce0-6f0de44f7080/lib/python3.10/site-packages/clarifai/client/input.py\input.py]8;;\:]8;id=293937;file:///local_disk0/.ephemeral_nfs/envs/pythonEnv-07884fc6-4bdc-4cb9-bce0-6f0de44f7080/lib/python3.10/site-packages/clarifai/client/input.py#631\631]8;;\
                             description: "Duplicate URL in your application. Check the documentation              
                             to allow duplications."                                                               
                             details: "The input URL is duplicate."                                                
                                                                                                                   

Uploading inputs: 100%|██████████| 1/1 [00:00<00:00,  4.48it/s]


### Using a custom dataloader to upload images from Volume

In [ ]:
dataset_obj.upload_dataset_from_dataloader(task="visual-classification", split="train", module_dir="/Volumes/mansi_test/default/test_vol1/food-101")

2023-10-23 09:06:51 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=977950;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=507837;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading Dataset: 100%|██████████| 1/1 [00:02<00:00,  2.61s/it]


### Creating ClarifaiPyspark object & creating/fetching text dataset from app

In [ ]:
cspark_obj = ClarifaiPySpark(user_id='mansi_k', app_id='databricks_tester_txt')

dataset_obj = cspark_obj.dataset(dataset_id='dataset1')

2023-10-23 08:33:36 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=643398;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=913909;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Creating a new dataset


                    INFO     INFO:clarifai.client.app:                                                   ]8;id=28864;file:///local_disk0/.ephemeral_nfs/envs/pythonEnv-07884fc6-4bdc-4cb9-bce0-6f0de44f7080/lib/python3.10/site-packages/clarifai/client/app.py\app.py]8;;\:]8;id=868935;file:///local_disk0/.ephemeral_nfs/envs/pythonEnv-07884fc6-4bdc-4cb9-bce0-6f0de44f7080/lib/python3.10/site-packages/clarifai/client/app.py#291\291]8;;\
                             Dataset created                                                                       
                             code: SUCCESS                                                                         
                             description: "Ok"                                                                     
                             req_id: "ad013b11e5cf2894d78d3a5dfc368202"                                            
                                                                                                                   

### Fetching current inputs from text dataset

In [ ]:
my_inputs = dataset_obj.list_inputs()

my_inputs

2023-10-23 08:34:24 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=537671;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=861851;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

<generator object Inputs.list_inputs at 0x7f92a88c6570>

### Testing export_text_to_volume()

Q- Create folders acc to concept name?   
Q- Save text as csv?

In [ ]:
dataset_obj.export_text_to_volume(path="/Volumes/mansi_test/default/test_vol1", input_response=my_inputs)

2023-10-23 08:34:45 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=654716;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=492410;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Upload text from volume csv to Clarifai dataset

In [ ]:
dataset_obj.upload_dataset_from_csv(csv_path='/Volumes/mansi_test/default/test_vol1/emotions_data1.csv', input_type='text', labels=True, csv_type='raw')

2023-10-23 08:38:10 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=990114;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=279494;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading inputs: 100%|██████████| 1/1 [00:20<00:00, 20.56s/it]


### Load test csv file & upload text data into app using dataframe

In [ ]:
spark = SparkSession.builder.appName("Clarifai-spark").getOrCreate()
df = spark.read.option("header",True).csv("/Volumes/mansi_test/default/test_vol1/emotions_data1.csv")
df.show()

2023-10-23 08:38:54 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=298316;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=912974;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

+-------+--------------------+--------+
|inputid|               input|concepts|
+-------+--------------------+--------+
|    t11|   I am feeling good|   happy|
|    t21|This is a guava tree| neutral|
|    t31|This is a saddeni...|     sad|
+-------+--------------------+--------+



In [ ]:
dataset_obj.upload_dataset_from_dataframe(dataframe=df, input_type='text', df_type='raw', labels=True)

2023-10-23 08:39:47 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=241980;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=257321;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading inputs:   0%|          | 0/1 [00:00<?, ?it/s]

2023-10-23 08:39:48 WARNING  WARNING:clarifai.client.input:code: INPUT_INVALID_ARGUMENT                ]8;id=620274;file:///local_disk0/.ephemeral_nfs/envs/pythonEnv-07884fc6-4bdc-4cb9-bce0-6f0de44f7080/lib/python3.10/site-packages/clarifai/client/input.py\input.py]8;;\:]8;id=60565;file:///local_disk0/.ephemeral_nfs/envs/pythonEnv-07884fc6-4bdc-4cb9-bce0-6f0de44f7080/lib/python3.10/site-packages/clarifai/client/input.py#631\631]8;;\
                             description: "Input invalid argument"                                                 
                             details: "Input has a duplicate ID."                                                  
                                                                                                                   

Uploading inputs: 100%|██████████| 1/1 [00:00<00:00,  5.45it/s]
